In [1]:
import skimage.io as skio
import skimage
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import tifffile
import numpy as np
import os
from pathlib import Path
import trackpy as tp
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import glob
import matplotlib.animation as animation
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

In [ ]:
file_path = r"C:\Users\lizau\Desktop\walker_test\checking the movies"
in_files = glob.glob(f'{file_path}/*.tif')
cmap_bright = plt.colormaps.get_cmap("Set1")
out_dir = Path(f'{file_path}/out_2')

os.makedirs(out_dir, exist_ok=True)

In [3]:
#scale_bar_pixels = 1
pixelsize = 0.072 
scale_bar_length = 1
scale_bar_pixels = int(scale_bar_length / pixelsize)
range_min = 1000
range_max = 1900 

In [4]:
for file in in_files:
    basename = os.path.basename(file)
    basename_noext, ext = os.path.splitext(basename)
    tray_path = Path(f'{file_path}/{basename_noext}_link2_traylen3.tray.csv')
    tray = pd.read_csv(tray_path)
    stack = skio.imread(file)

    fig = plt.figure("FRAMES", dpi=300, frameon=False)
    ax = fig.add_subplot(111)
    frame_for_shape = ax.imshow(stack[0].astype('uint16'), cmap = 'Greys_r')
    image_array = frame_for_shape.get_array().data  # Retrieve the image array
    image_height, image_width = image_array.shape[:2]
    subset = stack[range_min:range_max+1]
    v_min = np.min(subset) #probably 0 for all cases anyway
    #v_max = np.max(subset)
    v_max = np.mean(np.partition(subset.ravel(), -100)[-100:])
    
    set1_dict = { particle:cmap_bright(i%9) for i, particle in enumerate(list(set(tray.particle)))}
    ims = [] 

    for n in range(range_min, range_max):
        frame = ax.imshow(stack[n].astype('uint16'), cmap = 'Greys_r', vmin = v_min, vmax = v_max)
        number = ax.annotate(f'{n*0.25} s',(1,5), color='white', fontsize='16')
        scale_bar = Rectangle(
            (image_width - 15, image_height - 4),  # Position (10 pixels from left, 20 pixels from bottom)
            scale_bar_pixels,  # Width of the scale bar in pixels
            1,  # Height of the scale bar in pixels
            color='white'
        )
        ax.add_patch(scale_bar)

        # Add text for the scale bar length
        scale_text = ax.text(
            image_width - 12, image_height - 5,  # Position slightly above the scale bar
            f'{scale_bar_length} µm',
            color='white',
            fontsize=10
        )

        artist_obj = [frame, number, scale_bar, scale_text]

        artist_obj.append(frame)
        artist_obj.append(number)
        artist_obj.append(scale_bar)
        artist_obj.append(scale_text)
        if n in list(set(tray.frame)):
            for k in list(set(tray[(tray["frame"]==n)].particle)):
                bright_color = set1_dict[k]
                line, = ax.plot(  # Note the comma to unpack the single Line2D object
                    tray[(tray["frame"] <= n) & (tray["particle"] == k)].x,
                    tray[(tray["frame"] <= n) & (tray["particle"] == k)].y,
                    linewidth=1,
                    color=bright_color
                )
                plt.axis('off')
            
                artist_obj.append(line)

        ims.append(artist_obj)
        #if n >= 50:
            #break

    ani = animation.ArtistAnimation(fig, ims, interval=500, blit=True,repeat_delay=500)

    output_path = f'{out_dir}/{basename_noext}_trajectories.mp4'

    ani.save(output_path)
    plt.close(fig)